In [ ]:
### Ejemeplo de T-Test de una muestra
### Ejemeplo de T-Test de dos muestra
### Ejemeplo de T-Test pareado
## ANova
## Independencia de datos cualitativos

In [ ]:
## Librerias
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import math

In [ ]:
np.random.seed(1234)

In [ ]:
poblacion1=ss.poisson.rvs(loc=18,mu=35,size=450000)
poblacion2=ss.poisson.rvs(loc=18,mu=15,size=150000)

In [ ]:
Espanya=np.concatenate((poblacion1,poblacion2))

In [ ]:
granada1=ss.poisson.rvs(loc=18,mu=34,size=4500)
granada2=ss.poisson.rvs(loc=18,mu=14,size=1500)
Granada=np.concatenate((granada1,granada2))

In [ ]:
print(Espanya.mean())
print(Granada.mean())

$$H_0 :   \mu_{G}=\mu_{ES}$$
$$H_1 : \mu_{G} \neq \mu_{ES}$$

In [ ]:
estadistico,pvalor=ss.ttest_1samp(Granada,Espanya.mean())

In [ ]:
if pvalor<=0.05:
    print("Rechazamos la ****")
else:
    print("Aceptamos la ***")

In [ ]:
Madrid1=ss.poisson.rvs(loc=18,mu=37,size=120)
Madrid2=ss.poisson.rvs(loc=18,mu=13,size=80)
Madrid=np.concatenate((Madrid1,Madrid2))

In [ ]:
Madrid.mean()

In [ ]:
ss.ttest_ind(Granada,Madrid,equal_var=False)

$$ H_0 : \mu_{Gr} \leq \mu_{Ma}$$
$$ H_1 : \mu_{Gr} > \mu_{Ma}$$

In [ ]:
ss.ttest_ind(Granada,Madrid,equal_var=False,alternative="greater")

In [ ]:
## T-test Pareado
peso_precovid=ss.norm.rvs(scale=30,loc=100,size=1000)
peso_postcovid=peso_precovid+ss.norm.rvs(scale=10,loc=1.25,size=1000)

In [ ]:
Pesos=pd.DataFrame({"peso_precovid":peso_precovid,
              "peso_postcovid":peso_postcovid,
              "Diferencia":peso_postcovid-peso_precovid})

In [ ]:
Pesos.info()

In [ ]:
Pesos.describe().T

$$ H_0: \mu_{AC} = \mu_{DC} $$
$$ H_1: \mu_{AC} \neq \mu_{DC} $$

In [ ]:
ss.ttest_rel(peso_precovid,peso_postcovid,alternative="two-sided")

$$ H_0: \mu_{AC} \leq \mu_{DC} $$
$$ H_1: \mu_{AC} > \mu_{DC} $$

In [ ]:
ss.ttest_rel(peso_precovid,peso_postcovid,alternative="greater")

$$ H_0: \mu_{AC} \geq \mu_{DC} $$
$$ H_1: \mu_{AC} < \mu_{DC} $$

In [ ]:
ss.ttest_rel(peso_precovid,peso_postcovid,alternative="less")

In [ ]:
ss.ttest_rel?

### Anova

In [ ]:
Peliculas=["IJ","Bat","Marvel","Senor_Anillos","Tenet"]
pelicula_vista=np.random.choice(a=Peliculas,
                                p=[0.1,0.2,0.2,0.15,0.35],
                                size=1000)
voto_Edad=ss.poisson.rvs(loc=9,mu=25,size=1000)

In [ ]:
df=pd.DataFrame({"Pelicula":pelicula_vista,
              "Edad":voto_Edad})

In [ ]:
df

In [ ]:
grupos=df.groupby('Pelicula').groups

In [ ]:
Bat=voto_Edad[grupos["Bat"]]
IJ=voto_Edad[grupos["IJ"]]
Marvel=voto_Edad[grupos["Marvel"]]
Tenet=voto_Edad[grupos["Tenet"]]
Senor_Anillos=voto_Edad[grupos["Senor_Anillos"]]

In [ ]:
ss.f_oneway(Senor_Anillos,Bat,Tenet,IJ,Marvel)

$$ H_0: \mu_{0}=\mu_{1}=\ldots=\mu_{n}$$
$$ H_1: \mu_{i}\neq\mu_{j} \text{existe un i que la media es diferente para un j}$$

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
modelo=ols('Edad~Pelicula',data=df).fit()

In [ ]:
modelo

In [ ]:
sm.stats.anova_lm(modelo,typ=2)

In [ ]:
IJ_Edad=ss.poisson.rvs(loc=18,mu=35,size=1000)
voto_Edad=np.where(pelicula_vista=="IJ",IJ_Edad,voto_Edad)

In [ ]:
df_1=pd.DataFrame({"Pelicula":pelicula_vista,
              "Edad":voto_Edad})

In [ ]:
modelo=ols('Edad~Pelicula',data=df_1).fit()

In [ ]:
sm.stats.anova_lm(modelo,typ=2)

### Ejercicio
De manera automática encontrar el grupo diferente

### Solución
* Hacer las parejas
    *  Crearé una lista vacía para guardar las parejas
    * Lo haré mediante un for y tengo que evitar que las parejas se repitan o sean de la misma pelicula
* Cada pareja pasará por un ttest_ind
* Supondré que la varianza es desigual
* Si el p>0.05 pasaré
* Si es P<0.05 me quedaré con esa pareja



In [ ]:
Peliculas=["IJ","Bat","Marvel","Senor_Anillos","Tenet"]
## Hacemos parejas
Parejas=[]
for x in range(4):
    for y in range(x+1,5):
        Parejas.append((Peliculas[x],Peliculas[y]))

In [ ]:
Parejas

In [ ]:
Parejas[0][0]

In [ ]:
ss.ttest_ind(voto_Edad[grupos[Parejas[0][0]]],
             voto_Edad[grupos[Parejas[0][1]]],
             equal_var=False)

In [ ]:
for Peli1,Peli2 in Parejas:
    estadistico,pvalor=ss.ttest_ind(voto_Edad[grupos[Peli1]],
             voto_Edad[grupos[Peli2]],
             equal_var=False)
    if pvalor<0.05:
        print("\n",Peli1,"-vs-",Peli2)

In [ ]:
from  statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
tukey=pairwise_tukeyhsd(voto_Edad,pelicula_vista,alpha=0.05)

In [ ]:
tukey.summary()

In [ ]:
tukey.plot_simultaneous()

## Datos Cualitativos

### Test de $\chi ^2 $
$$\sum \cfrac{(\text{Observado}-\text{Esperados})^2}{\text{Esperados}}$$

In [5]:
import pandas as pd
Nacional=pd.DataFrame(["Mates"]*10000 + ["EF"]*60000 +["Lengua"]*50000 + 
                      ["Idiomas"]*15000 + ["Otras"]*35000)

In [6]:
Nacional

,0
0,Mates
1,Mates
2,Mates
3,Mates
4,Mates
...,...
169995,Otras
169996,Otras
169997,Otras
169998,Otras


In [7]:
Madrid=pd.DataFrame(["Mates"]*600 + ["EF"]*300 +["Lengua"]*250 + 
                      ["Idiomas"]*75 + ["Otras"]*150)

In [9]:
Tabla_Nacional=pd.crosstab(index=Nacional[0],columns="Recuento")
Tabla_Madrid=pd.crosstab(index=Madrid[0],columns="Recuento")

In [10]:
Observado=Tabla_Madrid
Ratios=Tabla_Nacional/len(Nacional)

In [11]:
Ratios

col_0,Recuento
0,
EF,0.352941
Idiomas,0.088235
Lengua,0.294118
Mates,0.058824
Otras,0.205882


In [12]:
Esperado=Ratios*len(Madrid)

In [13]:
Esperado

col_0,Recuento
0,
EF,485.294118
Idiomas,121.323529
Lengua,404.411765
Mates,80.882353
Otras,283.088235


In [14]:
Observado

col_0,Recuento
0,
EF,300
Idiomas,75
Lengua,250
Mates,600
Otras,150


In [18]:
((Observado-Esperado)**2/Esperado).sum()

col_0
Recuento    3541.753247
dtype: float64

In [20]:
import scipy.stats as ss

### Opciones
    a. cdf +1
    b. sf +4
    c. pmd +1
    d. rvs
    e. ppf
    f. pdf +1

In [22]:
alpha=0.05
valor_Critico=ss.chi2.ppf(q=1-alpha,df=5-1)

3541.753246753247

In [24]:
if valor_Critico<((Observado-Esperado)**2/Esperado).sum()[0]:
    print("Rechazo la hipotesis nula")

Rechazo la hipotesis nula


In [25]:
ss.chisquare(Observado,Esperado)

Power_divergenceResult(statistic=array([3541.75324675]), pvalue=array([0.]))

In [26]:
p_Valor=ss.chi.sf(((Observado-Esperado)**2/Esperado).sum()[0],df=4)

In [27]:
p_Valor

0.0

In [28]:
import numpy as np
voto_Pais=np.random.choice(a=["Italia","Venezuela","Argentina","Uruguay","Israel"],
                           p=[0.15,0.15,0.25,0.35,0.1],
                           size=10000)
voto_Cantante=np.random.choice(a=["Cantante_X","Cantante_Y","Cantante_Z"],
                           p=[0.45,0.45,0.1],
                           size=10000)

In [29]:
votos=pd.DataFrame({"Pais":voto_Pais,
                    "Cantante":voto_Cantante})

In [30]:
votos

,Pais,Cantante
0,Venezuela,Cantante_X
1,Italia,Cantante_Y
2,Israel,Cantante_X
3,Uruguay,Cantante_Y
4,Uruguay,Cantante_X
...,...,...
9995,Venezuela,Cantante_X
9996,Uruguay,Cantante_X
9997,Uruguay,Cantante_X
9998,Argentina,Cantante_Z


In [32]:
votos_tab=pd.crosstab(voto_Pais,voto_Cantante,margins=True,margins_name="Total")

In [33]:
votos_tab

col_0,Cantante_X,Cantante_Y,Cantante_Z,Total
row_0,,,,
Argentina,1109,1126,241,2476
Israel,479,429,102,1010
Italia,730,640,130,1500
Uruguay,1557,1625,350,3532
Venezuela,657,682,143,1482
Total,4532,4502,966,10000


In [69]:
pablo=pd.DataFrame()
i=0
for x in  votos_tab["Total"]:
    j=0
    for y in votos_tab.loc["Total"]:
        pablo.loc[i,j]=x*y//len(voto_Pais)
        j+=1
    i+=1

In [70]:
pablo

,0,1,2,3
0,1122.0,1114.0,239.0,2476.0
1,457.0,454.0,97.0,1010.0
2,679.0,675.0,144.0,1500.0
3,1600.0,1590.0,341.0,3532.0
4,671.0,667.0,143.0,1482.0
5,4532.0,4502.0,966.0,10000.0


In [37]:
Esperados=np.outer(votos_tab["Total"][0:5],votos_tab.loc["Total"][0:3])/len(voto_Pais)

In [43]:
Esperados_df=pd.DataFrame(Esperados)
Esperados_df.index=["Italia","Venezuela","Argentina","Uruguay","Israel"]
Esperados_df.columns=["Cantante_X","Cantante_Y","Cantante_Z"]

In [44]:
Esperados_df

,Cantante_X,Cantante_Y,Cantante_Z
Italia,1122.1232,1114.6952,239.1816
Venezuela,457.7320,454.7020,97.5660
Argentina,679.8000,675.3000,144.9000
Uruguay,1600.7024,1590.1064,341.1912
Israel,671.6424,667.1964,143.1612


In [45]:
Observado=pd.crosstab(voto_Pais,voto_Cantante,margins=False)

In [55]:
Observado.index=["Italia","Venezuela","Argentina","Uruguay","Israel"]
Observado

col_0,Cantante_X,Cantante_Y,Cantante_Z
Italia,1109,1126,241
Venezuela,479,429,102
Argentina,730,640,130
Uruguay,1557,1625,350
Israel,657,682,143


In [54]:
Esperados_df

,Cantante_X,Cantante_Y,Cantante_Z
Italia,1122.1232,1114.6952,239.1816
Venezuela,457.7320,454.7020,97.5660
Argentina,679.8000,675.3000,144.9000
Uruguay,1600.7024,1590.1064,341.1912
Israel,671.6424,667.1964,143.1612


In [56]:
((Observado-Esperados_df)**2/Esperados_df).sum().sum()

12.843042037740018

In [57]:
valor_Critico=ss.chi2.ppf(0.95,df=(5-1)*(3-1))

In [58]:
valor_Critico

15.50731305586545

In [53]:
ss.chi2_contingency(Observado)

(12.84304203774002,
 0.11736535894672082,
 8,
 array([[1122.1232, 1114.6952,  239.1816],
        [ 457.732 ,  454.702 ,   97.566 ],
        [ 679.8   ,  675.3   ,  144.9   ],
        [1600.7024, 1590.1064,  341.1912],
        [ 671.6424,  667.1964,  143.1612]]))